##Mounting drive

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# %cd /content/drive/My\ Drive/Colab Notebooks/Data_mining/Lab1/Notebooks
# !ls

/content/drive/My Drive/Colab Notebooks/Data_mining/Lab1/Notebooks
apriori.ipynb  fp_Growth.ipynb


##Importing Libraries

In [10]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import copy
import math
import csv
import time
import re
import tracemalloc
import matplotlib.pyplot as plt

plt.rc('xtick',labelsize=15)
plt.rc('ytick',labelsize=15)

# Node class

In [11]:
class Node:
  def __init__(self, parent= None, val = None, remainingDepth = -1):
    self.parent = parent
    self.val = val
    self.children = {}
    self.supCnt = 0
    self.remainingDepth = remainingDepth
    if parent == None:
      self.level = 0
    else:
      self.level = parent.level+1
      parent.insertChild(self)

  def insertChild(self, node):
    self.children[node.val] = node

  def deleteChild(self, node):
    del self.children[node.val]
  

# Apriori implementation

In [12]:
bucket = np.array([])
def printPathForLevel(node, tLevel):
  global bucket
  if node.level==tLevel:
    print(bucket,' last element supCnt = ', node.supCnt)
  else:
    for child in node.children.values():
      bucket = np.append(bucket,child.val)
      # print('node.val=', node.val, ' child=',child.val,' tLevel=',tLevel )
      printPathForLevel(node.children[child.val], tLevel)
      bucket = bucket[:-1]

def printFullTrie(node, tLevel):
  print('Level:',tLevel,' printing full trie*************************')
  for tl in range(tLevel):
    printPathForLevel(node, tl+1)
  print('Level:',tLevel,' end printing full trie*************************')
    

In [13]:
def findInTrie(node, path, curInd, skipInd):
  if node.remainingDepth + node.level< len(path)-1:
    return False
  if curInd>=len(path):
    return True
  if curInd == skipInd:
    return findInTrie(node, path, curInd+1, skipInd)
  chKeys = list(node.children)
  for i in range(len(chKeys)):
    child = node.children[chKeys[i]]
    if child.val == path[curInd]:
      return findInTrie(child, path, curInd+1, skipInd)
  return False
def isPruned(node, path):
  # print('path=',path)
  for i in range(len(path)-2):
    if not findInTrie(node, path, 0 , i):
      return True
  return False

In [14]:
path = np.array([])
def generateCandidates(node, remDepth, level):
  global root
  global path
  totalCandidates = 0
  mxRemaingDepth = -1
  if node.remainingDepth < remDepth:
    return 0
  chKeys = list(node.children)
  for i in range(len(chKeys)):
    child = node.children[chKeys[i]]
    path = np.append(path, child.val)
    if child.level==level-1 and not child.children:
      mxRemaingDepthForChild = -1
      for j in range(i+1, len(chKeys)):
        path = np.append(path, node.children[chKeys[j]].val)
        if level<=2 or not isPruned(root, path):
          newNode = Node(child, node.children[chKeys[j]].val, 0)
          mxRemaingDepthForChild = max(mxRemaingDepthForChild, newNode.remainingDepth)
          totalCandidates += 1
        path = path[:-1]
      child.remainingDepth = mxRemaingDepthForChild + 1
    else:
      totalCandidates += generateCandidates(child, remDepth-1, level)

    path = path[:-1]
    mxRemaingDepth = max(mxRemaingDepth, child.remainingDepth)
  
  node.remainingDepth = mxRemaingDepth + 1
  return totalCandidates


In [15]:
def updateSupportCnt(node, transaction, curInd, remLength):
  if remLength < 0 or node.remainingDepth < remLength:
    return

  if remLength == 0 and len(node.children) == 0:
    node.supCnt+=1
    return
  
  allowedInd = range(curInd, len(transaction) - remLength + 1)
  for ind in allowedInd:
    if transaction[ind] in node.children:
      updateSupportCnt(node.children[transaction[ind]], transaction, curInd+1, remLength-1)

  # if curInd>len(transaction)-remLength or len(transaction)<level or node.remainingDepth<remLength:
  #   return
  # if node.level == level and not node.children:
  #   node.supCnt += 1
  #   return
  # chKeys = list(node.children)
  # for i in range(len(chKeys)):
  #   child = node.children[chKeys[i]]
  #   # print('child.val = {}, transaction[curInd] = {}'.format(type(child.val), type(transaction[curInd])))
  #   if child.val == transaction[curInd]:
  #     updateSupportCnt(child, transaction, curInd+1, remLength-1, level)
  # updateSupportCnt(node, transaction, curInd+1, remLength, level)

In [16]:
def removeNonFrequentItemsets(node, minSup, remDepth, level):
  totalRemovedItem = 0
  if node.remainingDepth < remDepth:
    return 0
  mxRemainingDepth = -1
  chKeys = list(node.children)
  for i in range(len(chKeys)):
    child = node.children[chKeys[i]]
    if child.level == level and not child.children:
      if child.supCnt < minSup:
        node.deleteChild(child)
        totalRemovedItem +=1
      else:
        mxRemainingDepth = max(mxRemainingDepth, child.remainingDepth)
    else:
      totalRemovedItem += removeNonFrequentItemsets(child, minSup, remDepth-1, level)
      mxRemainingDepth = max(mxRemainingDepth, child.remainingDepth)
  node.remainingDepth = mxRemainingDepth+1
  return totalRemovedItem

In [17]:
def sortTransactions(transactions):
  for i in range(len(transactions)):
    transactions[i] = np.unique(transactions[i])
  return transactions

In [18]:
root = Node()
def runApriori(transactions, allItems, minSup):
  global root, bucket, path
  bucket = np.array([])
  path = np.array([])
  root = Node()
  totalFrequentItem = 0
  
  transactions = sortTransactions(transactions)
  items, freq = np.unique(allItems, return_counts=True)
  # print(items, freq)
  mxRemaingDepth=-1
  # inserting each unique item in root of trie
  for i in range(len(items)):
    if freq[i]>=minSup:
      node = Node(root, items[i],0)
      node.supCnt = freq[i]
      mxRemaingDepth= max(mxRemaingDepth, node.remainingDepth)
      totalFrequentItem += 1
  root.remainingDepth= max(root.remainingDepth, mxRemaingDepth+1)
  # print('Level 1 frequent itemsets:')
  # printFullTrie(root, 1)

  level = 2

  while True:
    totalCandidates = generateCandidates(root, level-1, level)
    # print('Level: ',level,' after candidates generated')
    # printFullTrie(root, level)

    # print('Level ',level, ' total candidate generated=', totalCandidates)

    if totalCandidates==0:
      break
    
    for transaction in transactions:
      updateSupportCnt(root, transaction, 0, level)

    # print('Level: ',level,' after updating support count')
    # printFullTrie(root, level)

    totalRemovedItem = removeNonFrequentItemsets(root, minSup, level, level)

    if totalCandidates-totalRemovedItem==0:
      break

    totalFrequentItem += totalCandidates-totalRemovedItem

    print('Level ',level, ' total Frequent itemsets generated=', totalCandidates-totalRemovedItem )

    print('Level: ',level,' after removing non frequent itemsets')
    printFullTrie(root, level)

    level += 1

  return totalFrequentItem

# Dataset Load

In [19]:
def loadDataset(datasetName, isNumeric = False, delimeter = ' '):
  filepath = datasetName
  # filepath = '/content/drive/MyDrive/Colab Notebooks/Data Mining Assignments/{}'.format(datasetName)
  transactions = []
  allItems = []
  totalTransactions = 0
  with open(filepath) as f:
    for i, line in enumerate(f):
      totalTransactions+=1
      transaction = line.strip().split(delimeter) 
      for item in transaction:
        allItems.append(item)
      if isNumeric:
        transactions.append(np.array(transaction, dtype = int))
      else:
        transactions.append(transaction)
      
  allItems = np.array(allItems, dtype=int) if isNumeric else allItems
  # print(transactions)
  # print(allItems)

  return transactions, allItems, totalTransactions



In [20]:
transactions, allItems, totalTransactions = loadDataset('sampleDataset_1.csv', False)
# print(transactions)
# print(allItems)
totalFrequentItem = runApriori(transactions=transactions, allItems=allItems, minSup=3)
print('Total frequent itemsets genereted = ',totalFrequentItem)

transactions, allItems, totalTransactions = loadDataset('sampleDataset_2.csv', False)
# print(transactions)
# print(allItems)
totalFrequentItem = runApriori(transactions=transactions, allItems=allItems, minSup=2)
print('Total frequent itemsets genereted = ',totalFrequentItem)

FileNotFoundError: [Errno 2] No such file or directory: 'sampleDataset_1.csv'

In [ ]:
transactions, allItems, totalTransactions = loadDataset(datasetName= 'tvseries.csv', isNumeric= False, delimeter= ',')
# print(transactions)
# print(allItems)
totalFrequentItem = runApriori(transactions=transactions, allItems=allItems, minSup=2)
print('Total frequent itemsets genereted = ',totalFrequentItem)

# Datasets

In [31]:
def printAndSave(datasetName,timeCllection, memoryCllection, minSupCnts):
  timeStr = ''
  memoryStr=''
  minSupCntStr=''
  for i in range(len(minSupCnts)):
    if i==0:
      minSupCntStr+='Method,'+str(minSupCnts[i])
    elif i == len(minSupCnts)-1:
      minSupCntStr+=','+str(minSupCnts[i])+'\n'
    else:
      minSupCntStr+=','+str(minSupCnts[i])
  
  timeStr+=minSupCntStr
  for i in range(len(timeCllection)):
    if i==0:
      timeStr+='Apriori,'+str(timeCllection[i])
    elif i == len(timeCllection)-1:
      timeStr+=','+str(timeCllection[i])+'\n'
    else:
      timeStr+=','+str(timeCllection[i])

  memoryStr+=minSupCntStr
  for i in range(len(memoryCllection)):
    if i==0:
      memoryStr+='Apriori,'+str(memoryCllection[i]/ 10**6)
    elif i == len(memoryCllection)-1:
      memoryStr+=','+str(memoryCllection[i]/ 10**6)+'\n'
    else:
      memoryStr+=','+str(memoryCllection[i]/ 10**6)

  print('timeString: ',timeStr)
  print('memoryString: ',memoryStr)
  filepath = datasetName+'.csv'
  with open(filepath,'w') as f:
    f.write(timeStr)
  filepath = datasetName+'.csv'
  with open(filepath,'w') as f:
    f.write(memoryStr)

## Chess

In [ ]:
timeCllection1= []
memoryCllection1= []
transactions, allItems, totalTransactions = loadDataset('chess.dat', True)

minRelativeSupCnts = np.linspace(0.95, 0.8, 5)
# minRelativeSupCnts = np.linspace(0.95, 0.9, 5)
minSupCnts = np.array([np.ceil(val * totalTransactions) for val in minRelativeSupCnts])
print(minSupCnts)

for minSupCnt in minSupCnts:

  tracemalloc.start()  
  start = time.time()

  totalFrequentItem = runApriori(transactions=transactions, allItems=allItems, minSup=minSupCnt)
  print('Total frequent itemsets genereted = ',totalFrequentItem)

  end = time.time()
  timeCllection1.append(end-start)
  print(end-start)

  current, peak = tracemalloc.get_traced_memory()
  memoryCllection1.append(peak)
  print(f"Current memory usage is {current / 10**6}MB; Peak was {peak / 10**6}MB")
  tracemalloc.stop()

[3037. 2917. 2797. 2677. 2557.]
Total frequent itemsets genereted =  77
4.0084168910980225
Current memory usage is 0.304074MB; Peak was 1.439784MB
Total frequent itemsets genereted =  389
18.200729370117188
Current memory usage is 0.43079MB; Peak was 1.439119MB
Total frequent itemsets genereted =  1386
61.90696930885315
Current memory usage is 0.917038MB; Peak was 1.441442MB
Total frequent itemsets genereted =  3674
169.4127790927887
Current memory usage is 2.028234MB; Peak was 2.029831MB
Total frequent itemsets genereted =  8227
386.9397521018982
Current memory usage is 4.20955MB; Peak was 4.210783MB


In [40]:
# printAndSave('chess',timeCllection1, memoryCllection1, np.linspace(0.95, 0.8, 5))

## mushroom

In [ ]:
timeCllection2= []
memoryCllection2= []
transactions, allItems, totalTransactions = loadDataset('mushroom.dat', True)

minRelativeSupCnts = np.linspace(0.60, 0.40, 5)
minSupCnts = np.array([np.ceil(val * totalTransactions) for val in minRelativeSupCnts])
print(minSupCnts)

for minSupCnt in minSupCnts:

  tracemalloc.start()  
  start = time.time()

  totalFrequentItem = runApriori(transactions=transactions, allItems=allItems, minSup=minSupCnt)
  print('Total frequent itemsets genereted = ',totalFrequentItem)

  end = time.time()
  timeCllection2.append(end-start)
  print(end-start)

  current, peak = tracemalloc.get_traced_memory()
  memoryCllection2.append(peak)
  print(f"Current memory usage is {current / 10**6}MB; Peak was {peak / 10**6}MB")
  tracemalloc.stop()

[4875. 4469. 4062. 3656. 3250.]
Total frequent itemsets genereted =  51
4.099918365478516
Current memory usage is 0.686164MB; Peak was 2.519727MB
Total frequent itemsets genereted =  99
6.8566694259643555
Current memory usage is 0.703477MB; Peak was 2.52205MB
Total frequent itemsets genereted =  153
9.55532431602478
Current memory usage is 0.710216MB; Peak was 2.519359MB
Total frequent itemsets genereted =  329
19.03514313697815
Current memory usage is 0.816473MB; Peak was 2.521994MB
Total frequent itemsets genereted =  565
28.016829013824463
Current memory usage is 0.930254MB; Peak was 2.521994MB


In [39]:
# printAndSave('mushroom',timeCllection2, memoryCllection2, np.linspace(0.60, 0.40, 5))

## retail

In [24]:
timeCllection3= []
memoryCllection3= []
transactions, allItems, totalTransactions = loadDataset('retail.dat', True)

minRelativeSupCnts = np.linspace(0.005, 0.003, 5)
# minRelativeSupCnts = np.linspace(0.95, 0.9, 5)
minSupCnts = np.array([np.ceil(val * totalTransactions) for val in minRelativeSupCnts])
print(minSupCnts)

for minSupCnt in minSupCnts:

  tracemalloc.start()  
  start = time.time()

  totalFrequentItem = runApriori(transactions=transactions, allItems=allItems, minSup=minSupCnt)
  print('Total frequent itemsets genereted = ',totalFrequentItem)

  end = time.time()
  timeCllection3.append(end-start)
  print(end-start)

  current, peak = tracemalloc.get_traced_memory()
  memoryCllection3.append(peak)
  print(f"Current memory usage is {current / 10**6}MB; Peak was {peak / 10**6}MB")
  tracemalloc.stop()

[441. 397. 353. 309. 265.]
Total frequent itemsets genereted =  580
15.379613399505615
Current memory usage is 62.589213MB; Peak was 116.650447MB
Total frequent itemsets genereted =  703
18.34506583213806
Current memory usage is 8.868341MB; Peak was 25.799567MB
Total frequent itemsets genereted =  831
22.203083038330078
Current memory usage is 9.31355MB; Peak was 31.763603MB
Total frequent itemsets genereted =  1075
32.9248263835907
Current memory usage is 10.672957MB; Peak was 45.824339MB
Total frequent itemsets genereted =  1393
54.990610122680664
Current memory usage is 12.993039MB; Peak was 72.123047MB


In [49]:
# printAndSave('retail',timeCllection3, memoryCllection3, np.linspace(0.005, 0.003, 5))

timeString:  Method,0.005,0.0045000000000000005,0.004,0.0035,0.003
Apriori,15.379613399505615,18.34506583213806,22.203083038330078,32.9248263835907,54.990610122680664

memoryString:  Method,0.005,0.0045000000000000005,0.004,0.0035,0.003
Apriori,116.650447,25.799567,31.763603,45.824339,72.123047



## T10I4D100K

In [26]:
timeCllection4= []
memoryCllection4= []
transactions, allItems, totalTransactions = loadDataset('T10I4D100K.dat', True)

minRelativeSupCnts = np.linspace(0.04, 0.02, 5)
minSupCnts = np.array([np.ceil(val * totalTransactions) for val in minRelativeSupCnts])
print(minSupCnts)

for minSupCnt in minSupCnts:

  tracemalloc.start()  
  start = time.time()

  totalFrequentItem = runApriori(transactions=transactions, allItems=allItems, minSup=minSupCnt)
  print('Total frequent itemsets genereted = ',totalFrequentItem)

  end = time.time()
  timeCllection4.append(end-start)
  print(end-start)

  current, peak = tracemalloc.get_traced_memory()
  memoryCllection4.append(peak)
  print(f"Current memory usage is {current / 10**6}MB; Peak was {peak / 10**6}MB")
  tracemalloc.stop()

[4000. 3501. 3000. 2500. 2000.]
Total frequent itemsets genereted =  26
3.127579927444458
Current memory usage is 8.046608MB; Peak was 18.104482MB
Total frequent itemsets genereted =  40
3.549589157104492
Current memory usage is 8.053568MB; Peak was 18.103442MB
Total frequent itemsets genereted =  60
4.22825288772583
Current memory usage is 8.104887MB; Peak was 18.103442MB
Total frequent itemsets genereted =  107
6.104845285415649
Current memory usage is 8.288721MB; Peak was 18.103514MB
Total frequent itemsets genereted =  155
7.758108377456665
Current memory usage is 8.519561MB; Peak was 18.10417MB


In [37]:
# printAndSave('T10I4D100K',timeCllection4, memoryCllection4, np.linspace(0.04, 0.02, 5))

## kosarak

In [28]:
timeCllection5= []
memoryCllection5= []
transactions, allItems, totalTransactions = loadDataset('kosarak.dat', True)

minRelativeSupCnts = np.linspace(0.15, 0.05, 5)
# minRelativeSupCnts = np.linspace(0.95, 0.9, 5)
minSupCnts = np.array([np.ceil(val * totalTransactions) for val in minRelativeSupCnts])
print(minSupCnts)

for minSupCnt in minSupCnts:

  tracemalloc.start()  
  start = time.time()

  totalFrequentItem = runApriori(transactions=transactions, allItems=allItems, minSup=minSupCnt)
  print('Total frequent itemsets genereted = ',totalFrequentItem)

  end = time.time()
  timeCllection5.append(end-start)
  print(end-start)

  current, peak = tracemalloc.get_traced_memory()
  memoryCllection5.append(peak)
  print(f"Current memory usage is {current / 10**6}MB; Peak was {peak / 10**6}MB")
  tracemalloc.stop()

[148501. 123751.  99001.  74251.  49501.]
Total frequent itemsets genereted =  7
38.25386118888855
Current memory usage is 89.377251MB; Peak was 169.560632MB
Total frequent itemsets genereted =  9
38.43591594696045
Current memory usage is 89.248139MB; Peak was 169.431376MB
Total frequent itemsets genereted =  9
38.017595529556274
Current memory usage is 89.247908MB; Peak was 169.431624MB
Total frequent itemsets genereted =  16
48.64606046676636
Current memory usage is 89.253028MB; Peak was 169.431632MB
Total frequent itemsets genereted =  33
82.8538269996643
Current memory usage is 89.264595MB; Peak was 169.430456MB


In [36]:
# printAndSave('kosarak',timeCllection5, memoryCllection5, np.linspace(0.15, 0.05, 5))